In [1]:
# load required packages
import geopandas as gpd
import rasterio
import pandas as pd

import numpy as np
import pickle
import rasterio.plot
from pathlib import Path

In [2]:
%store -r min_year
%store -r max_year
%store -r dam_forcast_working_dir
%store -r ANU_cliamte_downlaod_dir
%store -r  pkl_file_name_pred
%store -r  pkl_file_name_tavg
print(min_year, max_year)

2016 2023


In [3]:
limit_years = list(range(min_year, max_year + 1))
limit_years

[2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023]

In [4]:
dam_forcast_working_dir

PosixPath('/home/nick/Documents/Work code/Weather-to-water/working')

In [5]:
required_data_dir = dam_forcast_working_dir.parent / "Required data"
required_data_dir.mkdir(exist_ok=True)

In [6]:
data_output_dir = dam_forcast_working_dir.parent / "Data outputs"
data_output_dir.mkdir(exist_ok=True)

In [7]:
aus_states_path = required_data_dir / "States Map_cropped.shp"

dam_forcast_preds = pkl_file_name_pred
temp_data_path = pkl_file_name_tavg

In [8]:
check_point_data = False

In [9]:
AUSstates = gpd.read_file(aus_states_path)

In [10]:
AUSstates = AUSstates.to_crs("EPSG:4283")
AUSstates["State.ID"] = (
    pd.factorize(AUSstates["NAME"])[0] + 1
)  # Adding 1 to start IDs from 1 instead of 0

In [11]:
climate_zones_vector_path = required_data_dir / "clim-zones.gpkg"
climate_zones_vector = gpd.read_file(climate_zones_vector_path)

In [12]:
with open(dam_forcast_preds, "rb") as f:
    Data_gdf = pickle.load(f)
# recast fp64 to fp32
fp64_cols = Data_gdf.select_dtypes(include=["float64"]).columns
Data_gdf[fp64_cols] = Data_gdf[fp64_cols].astype(np.float32)
drop_cols = ["class", "class_name", "file_name_pred", "water_area"]
Data_gdf.drop(columns=drop_cols, inplace=True)

In [13]:
Data_gdf.head(5)

,file_name,dam_area,SRC_DATE,geometry,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,...,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00
0,arcgisonline_1286093.tif,1782.758057,20170126.0,POINT (146.65881 -36.09349),677.566772,840.408569,659.503784,525.416931,745.286377,865.782898,...,862.216431,809.222778,670.039917,933.242981,928.596802,890.365601,712.583374,705.275269,836.771301,699.068237
1,arcgisonline_1337387.tif,1917.350952,20151214.0,POINT (145.80580 -38.45117),989.949524,979.183960,968.603699,931.457397,875.900024,1128.018677,...,1075.012939,986.214050,939.212219,921.366577,1150.405884,1185.731079,1058.899902,1141.486572,1031.736572,1152.135254
2,arcgisonline_1702518.tif,2300.790039,20141024.0,POINT (145.69447 -41.09943),1155.011597,1260.526123,1300.596436,1372.891357,1506.764038,1152.403076,...,1272.070557,1267.041016,1097.546753,1155.979858,1095.693726,1213.050537,1509.002075,1448.021362,1393.711304,1505.985962
3,arcgisonline_1274916.tif,403.480804,20160112.0,POINT (141.22659 -36.64662),48.408882,35.025928,36.955124,54.295753,71.317192,81.144302,...,104.010971,97.298676,108.136131,147.315430,148.127228,169.204346,150.753738,130.708710,117.937454,120.220680
4,arcgisonline_704977.tif,1774.881104,20160502.0,POINT (149.83683 -32.87746),874.861694,762.450378,812.491089,817.687500,905.842651,552.856079,...,925.588379,879.336121,729.499817,742.824402,830.408325,767.549377,848.057800,737.685547,665.722656,821.412415


In [14]:
required_cols = ["file_name", "dam_area", "SRC_DATE", "geometry"]

# water_area', 'file_name_pred', 'class', 'class_name'


required_cols

['file_name', 'dam_area', 'SRC_DATE', 'geometry']

In [15]:
# remove col if they dont contain the limit_year in the heading or are not in the required_cols
for limit_year in limit_years:

    for col in Data_gdf.columns:
        if str(limit_year) in col:
            required_cols.append(col)
Data_gdf = Data_gdf[required_cols]
# Data_gdf = Data_gdf[
#     [
#         col
#         for col in Data_gdf.columns
#         if (str(limit_year) in col) or (col in required_cols)
#     ]
# ]
# Data_gdf.head()

In [16]:
climate_zones_vector.rename(columns={"DN": "ClimateCode"}, inplace=True)
# cast ClimateCode tp float
climate_zones_vector["ClimateCode"] = climate_zones_vector["ClimateCode"].astype(float)
climate_zones_vector

,ClimateCode,geometry
0,5.0,"POLYGON ((112.04291 -9.79563, 112.06789 -9.795..."
1,5.0,"POLYGON ((134.37699 -9.97051, 134.37699 -9.995..."
2,5.0,"POLYGON ((153.03868 -9.79563, 153.06366 -9.795..."
3,5.0,"POLYGON ((154.01299 -10.07044, 154.01299 -10.1..."
4,2.0,"POLYGON ((142.27136 -10.09542, 142.29634 -10.0..."
...,...,...
304,6.0,"POLYGON ((143.84524 -38.47519, 143.92018 -38.4..."
305,6.0,"POLYGON ((143.47050 -38.57512, 143.74531 -38.5..."
306,6.0,"POLYGON ((148.04224 -42.57227, 148.14217 -42.5..."
307,6.0,"POLYGON ((145.16929 -40.82351, 145.26922 -40.8..."


In [17]:
# intersect the data with the climate zones to make new col ClimateCode
Data_gdf = gpd.sjoin(Data_gdf, climate_zones_vector, how="left", predicate="intersects")
# drop the index_right column
Data_gdf.drop(columns="index_right", inplace=True)

In [18]:
Data_gdf.head()

,file_name,dam_area,SRC_DATE,geometry,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,...,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00,ClimateCode
0,arcgisonline_1286093.tif,1782.758057,20170126.0,POINT (146.65881 -36.09349),677.566772,840.408569,659.503784,525.416931,745.286377,865.782898,...,809.222778,670.039917,933.242981,928.596802,890.365601,712.583374,705.275269,836.771301,699.068237,4.0
1,arcgisonline_1337387.tif,1917.350952,20151214.0,POINT (145.80580 -38.45117),989.949524,979.183960,968.603699,931.457397,875.900024,1128.018677,...,986.214050,939.212219,921.366577,1150.405884,1185.731079,1058.899902,1141.486572,1031.736572,1152.135254,5.0
2,arcgisonline_1702518.tif,2300.790039,20141024.0,POINT (145.69447 -41.09943),1155.011597,1260.526123,1300.596436,1372.891357,1506.764038,1152.403076,...,1267.041016,1097.546753,1155.979858,1095.693726,1213.050537,1509.002075,1448.021362,1393.711304,1505.985962,6.0
3,arcgisonline_1274916.tif,403.480804,20160112.0,POINT (141.22659 -36.64662),48.408882,35.025928,36.955124,54.295753,71.317192,81.144302,...,97.298676,108.136131,147.315430,148.127228,169.204346,150.753738,130.708710,117.937454,120.220680,5.0
4,arcgisonline_704977.tif,1774.881104,20160502.0,POINT (149.83683 -32.87746),874.861694,762.450378,812.491089,817.687500,905.842651,552.856079,...,879.336121,729.499817,742.824402,830.408325,767.549377,848.057800,737.685547,665.722656,821.412415,5.0


In [19]:
# Remove farm dams without climate code
Data_gdf = Data_gdf.dropna(subset=["ClimateCode"])

# Save as a pickle file
if check_point_data:
    with open(required_data_dir / "full_preds_v9.pkl", "wb") as f:
        pickle.dump(Data_gdf, f)

# Print dimensions
print(Data_gdf.shape)

(1246417, 101)


In [20]:
Data_gdf

,file_name,dam_area,SRC_DATE,geometry,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,...,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00,ClimateCode
0,arcgisonline_1286093.tif,1782.758057,20170126.0,POINT (146.65881 -36.09349),677.566772,840.408569,659.503784,525.416931,745.286377,865.782898,...,809.222778,670.039917,933.242981,928.596802,890.365601,712.583374,705.275269,836.771301,699.068237,4.0
1,arcgisonline_1337387.tif,1917.350952,20151214.0,POINT (145.80580 -38.45117),989.949524,979.183960,968.603699,931.457397,875.900024,1128.018677,...,986.214050,939.212219,921.366577,1150.405884,1185.731079,1058.899902,1141.486572,1031.736572,1152.135254,5.0
2,arcgisonline_1702518.tif,2300.790039,20141024.0,POINT (145.69447 -41.09943),1155.011597,1260.526123,1300.596436,1372.891357,1506.764038,1152.403076,...,1267.041016,1097.546753,1155.979858,1095.693726,1213.050537,1509.002075,1448.021362,1393.711304,1505.985962,6.0
3,arcgisonline_1274916.tif,403.480804,20160112.0,POINT (141.22659 -36.64662),48.408882,35.025928,36.955124,54.295753,71.317192,81.144302,...,97.298676,108.136131,147.315430,148.127228,169.204346,150.753738,130.708710,117.937454,120.220680,5.0
4,arcgisonline_704977.tif,1774.881104,20160502.0,POINT (149.83683 -32.87746),874.861694,762.450378,812.491089,817.687500,905.842651,552.856079,...,879.336121,729.499817,742.824402,830.408325,767.549377,848.057800,737.685547,665.722656,821.412415,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246412,arcgisonline_1363308.tif,1318.117676,20160420.0,POINT (147.08716 -37.83705),639.004089,607.378052,537.088013,635.240112,490.926453,595.889343,...,576.668152,604.328125,643.961426,649.395264,501.776123,503.825775,470.636566,621.415344,811.633850,5.0
1246413,arcgisonline_542177.tif,899.002747,20151002.0,POINT (149.02065 -34.27568),307.467377,328.663269,336.402649,299.827972,269.576721,363.450867,...,386.698242,353.530212,460.240417,397.038910,360.278625,369.653625,419.796967,408.896576,402.586731,5.0
1246414,arcgisonline_1026840.tif,2827.576904,20150820.0,POINT (151.09756 -24.87151),1036.787354,1301.570557,1322.380615,1194.491333,1143.528564,1228.881348,...,1505.593506,1061.563843,1308.988892,954.689087,1213.755493,834.801880,555.568481,350.765442,942.434753,4.0
1246415,arcgisonline_817089.tif,9795.696289,20150202.0,POINT (146.51041 -36.35657),5990.866699,7552.419434,6199.013184,5450.937012,6306.955566,5909.651367,...,5881.017090,5166.090820,6221.470703,6832.887695,4632.944336,6480.630859,5269.216309,5738.416504,5179.333496,4.0


In [ ]:
with open(temp_data_path, "rb") as f:
    Temp = pickle.load(f)

In [22]:
# remove col if they dont contain the limit_year in the heading or are not in the required_cols
Temp = Temp[
    [col for col in Temp.columns if (str(limit_year) in col) or (col in required_cols)]
]
Temp.head()

,file_name,dam_area,SRC_DATE,geometry,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,...,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00
0,arcgisonline_1286093.tif,1782.758036,20170126.0,POINT (146.65881 -36.09349),23.931067,23.643499,22.697956,17.169930,12.745146,9.154373,...,20.308769,14.827796,9.910368,9.371562,8.775784,10.045410,13.097185,14.001484,19.558014,22.467890
1,arcgisonline_1337387.tif,1917.350912,20151214.0,POINT (145.80580 -38.45117),18.986082,18.957176,18.405102,15.588622,12.592820,9.476832,...,16.835918,14.049404,10.746694,9.874914,9.807578,10.204640,12.551055,12.570178,15.328485,16.990316
2,arcgisonline_1702518.tif,2300.790143,20141023.0,POINT (145.69447 -41.09943),17.308918,16.850088,15.583759,13.140615,10.532803,8.896879,...,14.034088,12.396546,9.456347,8.694619,9.189066,8.877248,10.180424,10.435368,13.774740,15.087011
3,arcgisonline_1274916.tif,403.480805,20160112.0,POINT (141.22659 -36.64662),22.228704,20.797312,20.297064,16.061920,13.358283,9.425863,...,17.394182,14.657558,11.197517,10.559347,9.646945,10.366800,12.950374,13.223871,16.688864,18.861603
4,arcgisonline_704977.tif,1774.881116,20160502.0,POINT (149.83683 -32.87746),20.285496,20.864256,19.654476,16.253624,11.375727,7.766718,...,19.202953,13.119492,8.176756,7.547411,7.969513,9.240471,12.535453,14.465531,17.863266,21.130512


In [23]:
Temp

,file_name,dam_area,SRC_DATE,geometry,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,...,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00
0,arcgisonline_1286093.tif,1782.758036,20170126.0,POINT (146.65881 -36.09349),23.931067,23.643499,22.697956,17.169930,12.745146,9.154373,...,20.308769,14.827796,9.910368,9.371562,8.775784,10.045410,13.097185,14.001484,19.558014,22.467890
1,arcgisonline_1337387.tif,1917.350912,20151214.0,POINT (145.80580 -38.45117),18.986082,18.957176,18.405102,15.588622,12.592820,9.476832,...,16.835918,14.049404,10.746694,9.874914,9.807578,10.204640,12.551055,12.570178,15.328485,16.990316
2,arcgisonline_1702518.tif,2300.790143,20141023.0,POINT (145.69447 -41.09943),17.308918,16.850088,15.583759,13.140615,10.532803,8.896879,...,14.034088,12.396546,9.456347,8.694619,9.189066,8.877248,10.180424,10.435368,13.774740,15.087011
3,arcgisonline_1274916.tif,403.480805,20160112.0,POINT (141.22659 -36.64662),22.228704,20.797312,20.297064,16.061920,13.358283,9.425863,...,17.394182,14.657558,11.197517,10.559347,9.646945,10.366800,12.950374,13.223871,16.688864,18.861603
4,arcgisonline_704977.tif,1774.881116,20160502.0,POINT (149.83683 -32.87746),20.285496,20.864256,19.654476,16.253624,11.375727,7.766718,...,19.202953,13.119492,8.176756,7.547411,7.969513,9.240471,12.535453,14.465531,17.863266,21.130512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246412,arcgisonline_1363308.tif,1318.117702,20160419.0,POINT (147.08716 -37.83705),20.471277,20.596754,19.842909,16.315517,13.282104,9.792249,...,18.704561,14.826365,11.104086,10.034918,10.051415,10.881070,13.740545,14.204240,17.187248,19.210045
1246413,arcgisonline_542177.tif,899.002759,20151002.0,POINT (149.02065 -34.27568),22.617380,22.846046,21.430828,16.953541,11.765292,8.835156,...,19.806530,14.176953,8.984231,8.568007,8.322306,9.401345,12.716261,14.522363,18.968554,21.960922
1246414,arcgisonline_1026840.tif,2827.576893,20150819.0,POINT (151.09756 -24.87151),26.211447,25.882643,25.016088,22.458158,19.174881,15.272620,...,25.356836,20.781359,15.160785,15.736105,14.768120,16.449141,19.095501,21.984009,24.141596,26.613298
1246415,arcgisonline_817089.tif,9795.696651,20150202.0,POINT (146.51041 -36.35657),23.173449,22.876051,21.989708,16.520969,12.309688,8.784438,...,19.661921,14.338357,9.583755,9.083367,8.599970,9.791127,12.780075,13.434568,18.937593,21.721962


In [24]:
# 2) Add the state where the points come from
# =============

# Transform AUSstates to match Data_gdf CRS
AUSstates = AUSstates.to_crs(Data_gdf.crs)

# Create a new raster extent based on Data_gdf
# bbox = Data_gdf.total_bounds  # [minx, miny, maxx, maxy]
# resolution = 0.001
# transform = rasterio.transform.from_origin(bbox[0], bbox[3], resolution, resolution)
# rows = int((bbox[3] - bbox[1]) / resolution)
# cols = int((bbox[2] - bbox[0]) / resolution)

# Rasterize AUSstates geometries
# from rasterio.features import rasterize

# shapes = (
#     (geom, value) for geom, value in zip(AUSstates.geometry, AUSstates["State.ID"])
# )
# with rasterio.open(
#     "AUSstates_raster.tif",
#     "w",
#     driver="GTiff",
#     height=rows,
#     width=cols,
#     count=1,
#     dtype="int32",
#     crs=Data_gdf.crs,
#     transform=transform,
# ) as dst:
#     dst_arr = rasterize(
#         shapes=shapes,
#         out_shape=(rows, cols),
#         transform=transform,
#         fill=0,
#         dtype="int32",
#     )
#     dst.write(dst_arr, 1)
# del dst_arr

In [25]:
# Extract State.ID for each point in Data_gdf
with rasterio.open(required_data_dir / "AUSstates_raster_comp.tif") as src:
    coords = [(x, y) for x, y in zip(Data_gdf.geometry.x, Data_gdf.geometry.y)]
    Data_gdf["State.ID"] = [val[0] for val in src.sample(coords)]

# Create the dictionary for state names
AUSstates_df = AUSstates[["NAME", "State.ID"]]

# Merge Data_gdf with state information
Data2 = Data_gdf.merge(AUSstates_df, on="State.ID", how="left")
del Data_gdf

# set State.ID  to uint8
Data2["State.ID"] = Data2["State.ID"].astype("uint8")
# ClimateCode to uint8
Data2["ClimateCode"] = Data2["ClimateCode"].astype("uint8")

# Check if the merge worked
print(Data2["NAME"].value_counts())

NAME
New South Wales                 392191
Victoria                        342869
Queensland                      208555
Western Australia               160822
South Australia                  85348
Tasmania                         55304
Australian Capital Territory      1257
Other Territories                   12
Name: count, dtype: int64


In [26]:
# Data2.dtypes

In [27]:
# Print dimensions
print(Data2.shape)
# Save Data2
if check_point_data:
    with open(data_output_dir / "Data2.pkl", "wb") as f:
        pickle.dump(Data2, f)

(1246417, 103)


In [28]:
Data2.shape

(1246417, 103)

In [29]:
Data2.columns

Index(['file_name', 'dam_area', 'SRC_DATE', 'geometry', '2016-01-01 00:00:00',
       '2016-02-01 00:00:00', '2016-03-01 00:00:00', '2016-04-01 00:00:00',
       '2016-05-01 00:00:00', '2016-06-01 00:00:00',
       ...
       '2023-06-01 00:00:00', '2023-07-01 00:00:00', '2023-08-01 00:00:00',
       '2023-09-01 00:00:00', '2023-10-01 00:00:00', '2023-11-01 00:00:00',
       '2023-12-01 00:00:00', 'ClimateCode', 'State.ID', 'NAME'],
      dtype='object', length=103)

In [30]:
# 3) Convert Data2 (m2) into data frame (remover NAT date and NAN Area)

# Convert Data2 into a DataFrame and select specific columns
Data3 = pd.DataFrame(Data2).drop(
    columns=[
        "State.ID",
        "SRC_DATE",
        "geometry",
    ]
)
del Data2

In [31]:
# Filter rows where all columns starting with a date have values greater than 0
Data3 = Data3[Data3.filter(regex="^\d{4}-\d{2}-\d{2}").gt(0).all(axis=1)]

# Pivot longer to convert wide format to long format
Data3 = Data3.melt(
    id_vars=["file_name", "dam_area", "NAME"], var_name="Date", value_name="Area"
)


# Convert Date column to datetime
Data3["Date"] = pd.to_datetime(Data3["Date"], errors="coerce")

# Remove rows where Date is NaT or Area is NaN/0
Data3 = Data3.dropna(subset=["Date"]).query("Area > 0")

In [32]:
# Add financial year column
Data3["FinYear"] = (Data3["Date"] + pd.offsets.MonthEnd(7)).dt.year.astype(str)

# Convert NAME to categorical
Data3["NAME"] = Data3["NAME"].astype("category")

In [33]:
Data3[Data3["FinYear"] == "2023"].head(20)

,file_name,dam_area,NAME,Date,Area,FinYear
97220526,arcgisonline_1286093.tif,1782.758057,Victoria,2022-07-01,643.141602,2023
97220527,arcgisonline_1337387.tif,1917.350952,Victoria,2022-07-01,1091.312134,2023
97220528,arcgisonline_1702518.tif,2300.790039,Tasmania,2022-07-01,1366.574951,2023
97220529,arcgisonline_1274916.tif,403.480804,Victoria,2022-07-01,126.766090,2023
97220530,arcgisonline_704977.tif,1774.881104,New South Wales,2022-07-01,927.035522,2023
97220531,arcgisonline_855690.tif,923.549683,Queensland,2022-07-01,364.908844,2023
97220532,arcgisonline_1040614.tif,684.269531,Queensland,2022-07-01,258.912201,2023
97220533,arcgisonline_589721.tif,832.694641,New South Wales,2022-07-01,384.944458,2023
97220534,arcgisonline_865858.tif,5366.125000,Queensland,2022-07-01,2581.144775,2023
97220535,arcgisonline_543794.tif,1014.401428,New South Wales,2022-07-01,413.722961,2023


In [34]:
# Save Data3
if check_point_data:
    with open(data_output_dir / "Data3.pkl", "wb") as f:
        pickle.dump(Data3, f)


# Print dimensions
Data3.shape, Data3.dtypes

((119656032, 6),
 file_name            object
 dam_area            float32
 NAME               category
 Date         datetime64[ns]
 Area                float32
 FinYear              object
 dtype: object)

In [35]:
Temp

,file_name,dam_area,SRC_DATE,geometry,2016-01-01 00:00:00,2016-02-01 00:00:00,2016-03-01 00:00:00,2016-04-01 00:00:00,2016-05-01 00:00:00,2016-06-01 00:00:00,...,2023-03-01 00:00:00,2023-04-01 00:00:00,2023-05-01 00:00:00,2023-06-01 00:00:00,2023-07-01 00:00:00,2023-08-01 00:00:00,2023-09-01 00:00:00,2023-10-01 00:00:00,2023-11-01 00:00:00,2023-12-01 00:00:00
0,arcgisonline_1286093.tif,1782.758036,20170126.0,POINT (146.65881 -36.09349),23.931067,23.643499,22.697956,17.169930,12.745146,9.154373,...,20.308769,14.827796,9.910368,9.371562,8.775784,10.045410,13.097185,14.001484,19.558014,22.467890
1,arcgisonline_1337387.tif,1917.350912,20151214.0,POINT (145.80580 -38.45117),18.986082,18.957176,18.405102,15.588622,12.592820,9.476832,...,16.835918,14.049404,10.746694,9.874914,9.807578,10.204640,12.551055,12.570178,15.328485,16.990316
2,arcgisonline_1702518.tif,2300.790143,20141023.0,POINT (145.69447 -41.09943),17.308918,16.850088,15.583759,13.140615,10.532803,8.896879,...,14.034088,12.396546,9.456347,8.694619,9.189066,8.877248,10.180424,10.435368,13.774740,15.087011
3,arcgisonline_1274916.tif,403.480805,20160112.0,POINT (141.22659 -36.64662),22.228704,20.797312,20.297064,16.061920,13.358283,9.425863,...,17.394182,14.657558,11.197517,10.559347,9.646945,10.366800,12.950374,13.223871,16.688864,18.861603
4,arcgisonline_704977.tif,1774.881116,20160502.0,POINT (149.83683 -32.87746),20.285496,20.864256,19.654476,16.253624,11.375727,7.766718,...,19.202953,13.119492,8.176756,7.547411,7.969513,9.240471,12.535453,14.465531,17.863266,21.130512
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1246412,arcgisonline_1363308.tif,1318.117702,20160419.0,POINT (147.08716 -37.83705),20.471277,20.596754,19.842909,16.315517,13.282104,9.792249,...,18.704561,14.826365,11.104086,10.034918,10.051415,10.881070,13.740545,14.204240,17.187248,19.210045
1246413,arcgisonline_542177.tif,899.002759,20151002.0,POINT (149.02065 -34.27568),22.617380,22.846046,21.430828,16.953541,11.765292,8.835156,...,19.806530,14.176953,8.984231,8.568007,8.322306,9.401345,12.716261,14.522363,18.968554,21.960922
1246414,arcgisonline_1026840.tif,2827.576893,20150819.0,POINT (151.09756 -24.87151),26.211447,25.882643,25.016088,22.458158,19.174881,15.272620,...,25.356836,20.781359,15.160785,15.736105,14.768120,16.449141,19.095501,21.984009,24.141596,26.613298
1246415,arcgisonline_817089.tif,9795.696651,20150202.0,POINT (146.51041 -36.35657),23.173449,22.876051,21.989708,16.520969,12.309688,8.784438,...,19.661921,14.338357,9.583755,9.083367,8.599970,9.791127,12.780075,13.434568,18.937593,21.721962


In [36]:
# 4) Convert Temp (C) into data frame

# Convert Temp into a DataFrame and select specific columns
Temp2 = pd.DataFrame(Temp).drop(
    columns=[
        "SRC_DATE",
    ]
)
del Temp
# set water_area to float32
Temp2["dam_area"] = Temp2["dam_area"].astype("float32")
# Pivot longer to convert wide format to long format
Temp2 = Temp2.melt(
    id_vars=["file_name", "dam_area", "geometry"],
    var_name="Date",
    value_name="Temp",
)
# Convert Date column to datetime
Temp2["Date"] = pd.to_datetime(
    Temp2["Date"], format="%Y-%m-%d %H:%M:%S", errors="coerce"
)

# Remove rows where Date is NaT or Temp is NaN
Temp2 = Temp2.dropna(subset=["Date", "Temp"])

# Add financial year column
Temp2["FinYear"] = (Temp2["Date"] + pd.offsets.MonthEnd(7)).dt.year.astype(str)

In [37]:
Temp2.head(20)

,file_name,dam_area,geometry,Date,Temp,FinYear
0,arcgisonline_1286093.tif,1782.758057,POINT (146.65881 -36.09349),2016-01-01,23.931067,2016
1,arcgisonline_1337387.tif,1917.350952,POINT (145.80580 -38.45117),2016-01-01,18.986082,2016
2,arcgisonline_1702518.tif,2300.790039,POINT (145.69447 -41.09943),2016-01-01,17.308918,2016
3,arcgisonline_1274916.tif,403.480804,POINT (141.22659 -36.64662),2016-01-01,22.228704,2016
4,arcgisonline_704977.tif,1774.881104,POINT (149.83683 -32.87746),2016-01-01,20.285496,2016
5,arcgisonline_855690.tif,923.549683,POINT (149.57990 -28.25137),2016-01-01,26.996311,2016
6,arcgisonline_1040614.tif,684.269531,POINT (152.42658 -26.24384),2016-01-01,25.028366,2016
7,arcgisonline_589721.tif,832.694641,POINT (153.10273 -28.99318),2016-01-01,24.227482,2016
8,arcgisonline_865858.tif,5366.125000,POINT (149.71875 -27.61193),2016-01-01,26.756281,2016
9,arcgisonline_543794.tif,1014.401428,POINT (147.95993 -34.59635),2016-01-01,23.583084,2016


In [38]:
# Save Temp2
if check_point_data:
    with open(data_output_dir / "Temp2.pkl", "wb") as f:
        pickle.dump(Temp2, f)


# Print dimensions
print(Temp2.shape)

(119656032, 6)


In [39]:
# 5) Merge Temp (C) and Area (m2)

# Filter Temp2 to exclude redundant data (years > 1979)
Temp2 = Temp2[Temp2["Date"].dt.year > 1979]

# Full join Data3 and Temp2
Data4 = pd.merge(Data3, Temp2, on=["file_name", "Date", "FinYear"], how="outer")
del Data3, Temp2

drop_cols = ["dam_area_y", "geometry", "file_name"]
Data4.drop(columns=drop_cols, inplace=True)
# Check the dimension
print(Data4.shape)

(119656032, 6)


In [40]:
Data4.head(10)

,dam_area_x,NAME,Date,Area,FinYear,Temp
0,31936.664062,Queensland,2016-01-01,9663.912109,2016,27.690578
1,31936.664062,Queensland,2016-02-01,8918.267578,2016,27.828859
2,31936.664062,Queensland,2016-03-01,9560.470703,2016,26.509073
3,31936.664062,Queensland,2016-04-01,10561.349609,2016,24.995647
4,31936.664062,Queensland,2016-05-01,8634.044922,2016,23.544092
5,31936.664062,Queensland,2016-06-01,8327.958008,2016,20.016394
6,31936.664062,Queensland,2016-07-01,10351.921875,2017,19.234936
7,31936.664062,Queensland,2016-08-01,9316.967773,2017,19.170479
8,31936.664062,Queensland,2016-09-01,6041.241699,2017,22.584221
9,31936.664062,Queensland,2016-10-01,9453.407227,2017,23.089096


In [41]:
# Save Data4
if check_point_data:
    with open(data_output_dir / "Data4.pkl", "wb") as f:
        pickle.dump(Data4, f)

In [42]:
# 6) Filter farm dams because either too big or too small
# ==============

# Filter Data4 to create Data5, excluding dams that are too big or too small
Data5 = Data4[(Data4["dam_area_x"] > 100) & (Data4["dam_area_x"] < 80000)].copy()

del Data4

# Print dimensions to verify filtering
print(Data5.shape)

(117205632, 6)


In [43]:
Data5.head(15)

,dam_area_x,NAME,Date,Area,FinYear,Temp
0,31936.664062,Queensland,2016-01-01,9663.912109,2016,27.690578
1,31936.664062,Queensland,2016-02-01,8918.267578,2016,27.828859
2,31936.664062,Queensland,2016-03-01,9560.470703,2016,26.509073
3,31936.664062,Queensland,2016-04-01,10561.349609,2016,24.995647
4,31936.664062,Queensland,2016-05-01,8634.044922,2016,23.544092
5,31936.664062,Queensland,2016-06-01,8327.958008,2016,20.016394
6,31936.664062,Queensland,2016-07-01,10351.921875,2017,19.234936
7,31936.664062,Queensland,2016-08-01,9316.967773,2017,19.170479
8,31936.664062,Queensland,2016-09-01,6041.241699,2017,22.584221
9,31936.664062,Queensland,2016-10-01,9453.407227,2017,23.089096


In [44]:
# Constants
boltzmann_constant = 8.617333e-05
temp_15C = 273.15 + 15


# Vectorized calculation function
def calculate_ef_vectorized(temp_series, coefficient, log_value):
    return (
        np.exp(
            coefficient
            * (
                (1 / (boltzmann_constant * temp_15C))
                - (1 / (boltzmann_constant * (273.15 + temp_series)))
            )
            + log_value
        )
        / 1000
    )


# Calculate all EFs at once
Data5["EF"] = calculate_ef_vectorized(
    Data5["Temp"], 0.4287063, np.log(365.25 * 10000 * (10**1.74705) / 1e6)
)

Data5["EF_LCI"] = calculate_ef_vectorized(
    Data5["Temp"], 0.2143579, np.log(365.25 * 10000 * (10**1.354771) / 1e6)
)

Data5["EF_UCI"] = calculate_ef_vectorized(
    Data5["Temp"], 0.6430547, np.log(365.25 * 10000 * (10**2.1546436) / 1e6)
)
Data5.head(5)

,dam_area_x,NAME,Date,Area,FinYear,Temp,EF,EF_LCI,EF_UCI
0,31936.664062,Queensland,2016-01-01,9663.912109,2016,27.690578,0.422609,0.118991,1.554811
1,31936.664062,Queensland,2016-02-01,8918.267578,2016,27.828859,0.425832,0.119444,1.572633
2,31936.664062,Queensland,2016-03-01,9560.470703,2016,26.509073,0.395933,0.115174,1.409948
3,31936.664062,Queensland,2016-04-01,10561.349609,2016,24.995647,0.363934,0.110422,1.242521
4,31936.664062,Queensland,2016-05-01,8634.044922,2016,23.544092,0.335404,0.106005,1.099318


In [45]:
# 8) Calculate total emissions
# ==============
# Units: t CH4 year-1
Data5["TotCH4"] = Data5["EF"] * Data5["Area"] / 10000  # Convert Area from m2 to ha
Data5["TotCH4_LCI"] = (
    Data5["EF_LCI"] * (Data5["Area"] - 0.45 * Data5["Area"]) / 10000
)  # Convert Area from m2 to ha
Data5["TotCH4_UCI"] = (
    Data5["EF_UCI"] * (Data5["Area"] + 0.45 * Data5["Area"]) / 10000
)  # Convert Area from m2 to ha
print(Data5.shape)
Data5.head(10)

(117205632, 12)


,dam_area_x,NAME,Date,Area,FinYear,Temp,EF,EF_LCI,EF_UCI,TotCH4,TotCH4_LCI,TotCH4_UCI
0,31936.664062,Queensland,2016-01-01,9663.912109,2016,27.690578,0.422609,0.118991,1.554811,0.408406,0.063246,2.178706
1,31936.664062,Queensland,2016-02-01,8918.267578,2016,27.828859,0.425832,0.119444,1.572633,0.379768,0.058588,2.033648
2,31936.664062,Queensland,2016-03-01,9560.470703,2016,26.509073,0.395933,0.115174,1.409948,0.378531,0.060562,1.954567
3,31936.664062,Queensland,2016-04-01,10561.349609,2016,24.995647,0.363934,0.110422,1.242521,0.384363,0.064141,1.902791
4,31936.664062,Queensland,2016-05-01,8634.044922,2016,23.544092,0.335404,0.106005,1.099318,0.289589,0.050339,1.376277
5,31936.664062,Queensland,2016-06-01,8327.958008,2016,20.016394,0.274120,0.095833,0.812237,0.228286,0.043895,0.980820
6,31936.664062,Queensland,2016-07-01,10351.921875,2017,19.234936,0.261965,0.093684,0.758816,0.271184,0.053339,1.139004
7,31936.664062,Queensland,2016-08-01,9316.967773,2017,19.170479,0.260984,0.093508,0.754558,0.243158,0.047917,1.019378
8,31936.664062,Queensland,2016-09-01,6041.241699,2017,22.584221,0.317638,0.103160,1.013141,0.191893,0.034277,0.887491
9,31936.664062,Queensland,2016-10-01,9453.407227,2017,23.089096,0.326876,0.104649,1.057661,0.309010,0.054411,1.449783


In [46]:
if check_point_data:
    # Save Data5 to a pickle file
    with open(data_output_dir / "Data5.pkl", "wb") as f:
        pickle.dump(Data5, f)

In [47]:
# Calculate relative capacity
RelCapacity = (
    Data5.groupby(["Date", "NAME", "FinYear"], observed=False)
    .agg(
        RelSurfaceArea=("Area", lambda x: (x / Data5.loc[x.index, "dam_area_x"]).mean())
    )
    .reset_index()
)

# Save RelCapacity
if check_point_data:
    with open(data_output_dir / "RelCapacity.pkl", "wb") as f:
        pickle.dump(RelCapacity, f)

# Summarised version
RelCapacity["FinYear"] = RelCapacity["FinYear"].astype(
    int
)  # Convert FinYear to numeric
RelCapacity.rename(
    columns={"NAME": "State"}, inplace=True
)  # Rename NAME column to State

RelCapacity_sum = (
    RelCapacity.groupby(["FinYear", "State"], observed=False)
    .agg(RelSurfaceArea=("RelSurfaceArea", "mean"))
    .reset_index()
)

# Save RelCapacity_sum
if check_point_data:
    with open(data_output_dir / "RelCapacity_sum.pkl", "wb") as f:
        pickle.dump(RelCapacity_sum, f)

In [48]:
RelCapacity_sum.head(20)

,FinYear,State,RelSurfaceArea
0,2016,Australian Capital Territory,0.472393
1,2016,New South Wales,0.364156
2,2016,Other Territories,0.487999
3,2016,Queensland,0.405820
4,2016,South Australia,0.224445
5,2016,Tasmania,0.355856
6,2016,Victoria,0.318994
7,2016,Western Australia,0.329208
8,2017,Australian Capital Territory,0.474912
9,2017,New South Wales,0.411563


In [49]:
Data5.shape

(117205632, 12)

In [50]:
# 9) Summarise by States and Months
# ==============
Data6 = (
    Data5.groupby(["NAME", "FinYear", "Date"], observed=True)
    .agg(
        SumArea_kha=("Area", lambda x: x.sum() * 1e-7),
        SumArea_kha_LCI=("Area", lambda x: (x.sum() * 1e-7) - 0.45 * (x.sum() * 1e-7)),
        SumArea_kha_UCI=("Area", lambda x: (x.sum() * 1e-7) + 0.45 * (x.sum() * 1e-7)),
        EF=("EF", "mean"),
        EF_LCI=("EF_LCI", "mean"),
        EF_UCI=("EF_UCI", "mean"),
        TotCH4_ktyear=("TotCH4", lambda x: x.sum() * 1e-3),
        TotCH4_ktyear_LCI=("TotCH4_LCI", lambda x: x.sum() * 1e-3),
        TotCH4_ktyear_UCI=("TotCH4_UCI", lambda x: x.sum() * 1e-3),
        Temp=("Temp", "mean"),
        Count=("TotCH4", "size"),
    )
    .reset_index()
)

del Data5

In [51]:
Data6.head(10)

,NAME,FinYear,Date,SumArea_kha,SumArea_kha_LCI,SumArea_kha_UCI,EF,EF_LCI,EF_UCI,TotCH4_ktyear,TotCH4_ktyear_LCI,TotCH4_ktyear_UCI,Temp,Count
0,Australian Capital Territory,2016,2016-01-01,0.194233,0.106828,0.281638,0.285279,0.097750,0.862685,0.055644,0.010465,0.244488,20.698828,1220
1,Australian Capital Territory,2016,2016-02-01,0.201883,0.111035,0.292730,0.290050,0.098565,0.884395,0.058829,0.010970,0.260680,20.987658,1220
2,Australian Capital Territory,2016,2016-03-01,0.215355,0.118445,0.312265,0.269085,0.094936,0.790264,0.058149,0.011264,0.248037,19.688541,1220
3,Australian Capital Territory,2016,2016-04-01,0.197788,0.108784,0.286793,0.209720,0.083814,0.543707,0.041613,0.009132,0.156678,15.455789,1220
4,Australian Capital Territory,2016,2016-05-01,0.192468,0.105858,0.279079,0.155830,0.072248,0.348238,0.030058,0.007656,0.097503,10.567602,1220
5,Australian Capital Territory,2016,2016-06-01,0.209043,0.114974,0.303112,0.129036,0.065743,0.262401,0.027069,0.007572,0.079954,7.547317,1220
6,Australian Capital Territory,2017,2016-07-01,0.180510,0.099281,0.261740,0.125983,0.064961,0.253142,0.022834,0.006463,0.066658,7.168846,1220
7,Australian Capital Territory,2017,2016-08-01,0.223515,0.122933,0.324096,0.127643,0.065388,0.258159,0.028618,0.008051,0.084051,7.375787,1220
8,Australian Capital Territory,2017,2016-09-01,0.202984,0.111641,0.294327,0.150388,0.070975,0.330150,0.030610,0.007935,0.097567,9.994007,1220
9,Australian Capital Territory,2017,2016-10-01,0.240053,0.132029,0.348077,0.165991,0.074564,0.382873,0.039975,0.009861,0.133906,11.592001,1220


In [52]:
# Save Data6
if check_point_data:
    with open(data_output_dir / "Data6.pkl", "wb") as f:
        pickle.dump(Data6, f)

# Print dimensions
Data6.head(5)  # Equivalent to dim(Data6) in R

,NAME,FinYear,Date,SumArea_kha,SumArea_kha_LCI,SumArea_kha_UCI,EF,EF_LCI,EF_UCI,TotCH4_ktyear,TotCH4_ktyear_LCI,TotCH4_ktyear_UCI,Temp,Count
0,Australian Capital Territory,2016,2016-01-01,0.194233,0.106828,0.281638,0.285279,0.097750,0.862685,0.055644,0.010465,0.244488,20.698828,1220
1,Australian Capital Territory,2016,2016-02-01,0.201883,0.111035,0.292730,0.290050,0.098565,0.884395,0.058829,0.010970,0.260680,20.987658,1220
2,Australian Capital Territory,2016,2016-03-01,0.215355,0.118445,0.312265,0.269085,0.094936,0.790264,0.058149,0.011264,0.248037,19.688541,1220
3,Australian Capital Territory,2016,2016-04-01,0.197788,0.108784,0.286793,0.209720,0.083814,0.543707,0.041613,0.009132,0.156678,15.455789,1220
4,Australian Capital Territory,2016,2016-05-01,0.192468,0.105858,0.279079,0.155830,0.072248,0.348238,0.030058,0.007656,0.097503,10.567602,1220


In [53]:
# Export as csv
Data6.to_csv(data_output_dir / "Data6.csv", index=False)

In [54]:
Data6.head(10)

,NAME,FinYear,Date,SumArea_kha,SumArea_kha_LCI,SumArea_kha_UCI,EF,EF_LCI,EF_UCI,TotCH4_ktyear,TotCH4_ktyear_LCI,TotCH4_ktyear_UCI,Temp,Count
0,Australian Capital Territory,2016,2016-01-01,0.194233,0.106828,0.281638,0.285279,0.097750,0.862685,0.055644,0.010465,0.244488,20.698828,1220
1,Australian Capital Territory,2016,2016-02-01,0.201883,0.111035,0.292730,0.290050,0.098565,0.884395,0.058829,0.010970,0.260680,20.987658,1220
2,Australian Capital Territory,2016,2016-03-01,0.215355,0.118445,0.312265,0.269085,0.094936,0.790264,0.058149,0.011264,0.248037,19.688541,1220
3,Australian Capital Territory,2016,2016-04-01,0.197788,0.108784,0.286793,0.209720,0.083814,0.543707,0.041613,0.009132,0.156678,15.455789,1220
4,Australian Capital Territory,2016,2016-05-01,0.192468,0.105858,0.279079,0.155830,0.072248,0.348238,0.030058,0.007656,0.097503,10.567602,1220
5,Australian Capital Territory,2016,2016-06-01,0.209043,0.114974,0.303112,0.129036,0.065743,0.262401,0.027069,0.007572,0.079954,7.547317,1220
6,Australian Capital Territory,2017,2016-07-01,0.180510,0.099281,0.261740,0.125983,0.064961,0.253142,0.022834,0.006463,0.066658,7.168846,1220
7,Australian Capital Territory,2017,2016-08-01,0.223515,0.122933,0.324096,0.127643,0.065388,0.258159,0.028618,0.008051,0.084051,7.375787,1220
8,Australian Capital Territory,2017,2016-09-01,0.202984,0.111641,0.294327,0.150388,0.070975,0.330150,0.030610,0.007935,0.097567,9.994007,1220
9,Australian Capital Territory,2017,2016-10-01,0.240053,0.132029,0.348077,0.165991,0.074564,0.382873,0.039975,0.009861,0.133906,11.592001,1220
